In [1]:
from llmp.services.job_storage import JobStorage
import os

job = JobStorage().get_job("fbe99910ff1f4f4e8e87940c72980ddc")

In [2]:
job.output_model

OutputModel(lines=[IOLine(key='external_types', type='list[str]', rule=None, options=['numeric', 'textual', 'multimedia', 'code'], multiple_select=True, default=None, placeholder=['external_types'], multiline=False, custom_value_template=None), IOLine(key='internal_types', type='list[str]', rule=None, options=['numeric', 'textual', 'multimedia', 'code'], multiple_select=True, default=None, placeholder=['internal_types'], multiline=False, custom_value_template=None)])

In [ ]:
from llmp.components.example_manager import ExampleManager

em = ExampleManager(job)

example_set = em.get_test_set(3)

In [ ]:
template = """Create the task instruction based on the input and output example provided below. The Input and Output Object are provided as a guide to the format of the input and output objects. The input and output objects may be different for each task but will have the same keys. So you should not include any information from the example given, instead you should use the input and output keys to describe the input and output objects.
Think step-by-step to reverse engineer the task instruction. What other inputs might look like with same keys? What is the relationship between them? What might be the right operation (label, extract, compress, generate, etc.) to get from any input with that structure to the output?

Remember do not include concrete values provided in Input Object and Output Object in your instruction. Instead, use the input and output model to describe the input and output objects so that your instruction can be applied to other inputs and outputs with the same model.


# Input
Input Model: {inp_model}
Output Model: {out_model}
Example Input Object: {input_object}
Example Output Output: {output_object}
---
Reasoning: <multiline>
Instructions: <multiline>
"""

In [ ]:
template = """Create the task instruction based on the input and output model provided below. The models define the input and output of the task you should create the instruction for.
Think step-by-step to reverse engineer the task instruction. What concrete inputs might look like with keys described in the input model? What is the relationship between them? What might be the right operation (label, extract, compress, generate, etc.) to get from any input with that structure to the output?


# Input
Input Model: {inp_model}
Output Model: {out_model}
---
Reasoning: <multiline>
Instructions: <multiline>
"""

In [ ]:
from structgenie.engine import StructGenie


engine = StructGenie.from_template(template, debug=True)
engine.run(
    dict(
        inp_model=job.input_model.template_schema,
        out_model=job.output_model.template_schema,
        input_object=None, # example_set[0].input,
        output_object=None, # example_set[0].output,
        num=5,
    )
)

In [ ]:
from structgenie.input_output.line import IOLine

from structgenie.input_output import OutputModel, InputModel
from structgenie.engine import StructGenie


engine = StructGenie.from_template(template, debug=True)
engine.run(
    dict(
        inp_model=InputModel(lines=[IOLine(key='book_title', type='str', rule=None, options=None, multiple_select=False, default=None, placeholder=['book_title'], multiline=False, custom_value_template=None), IOLine(key='book_author', type='str', rule=None, options=None, multiple_select=False, default=None, placeholder=['book_author'], multiline=False, custom_value_template=None), IOLine(key='release_year', type='int', rule=None, options=None, multiple_select=False, default=None, placeholder=['release_year'], multiline=False, custom_value_template=None)]).template_schema,
        out_model=OutputModel(lines=[IOLine(key='genre', type='str', rule=None, options=['fiction', 'non-fiction', 'fantasy', 'sci-fi', 'romance', 'thriller', 'horror', 'history'], multiple_select=False, default=None, placeholder=['genre'], multiline=False, custom_value_template=None)]).template_schema,
        input_object=None,
        output_object=None,
    )
)

In [ ]:
job.output_model.template_schema

In [ ]:
example_set[0].input_keys

In [8]:
from llmp.components.instruction.utils import generate_instruction_from_models, mutate_instruction, instruction_from_working_out, extend_instruction_by_model

In [6]:
from structgenie.input_output import OutputModel, InputModel

from structgenie.input_output.line import IOLine

input_model = InputModel(lines=[IOLine(key='book_title', type='str', rule=None, options=None, multiple_select=False, default=None, placeholder=['book_title'], multiline=False, custom_value_template=None), IOLine(key='book_author', type='str', rule=None, options=None, multiple_select=False, default=None, placeholder=['book_author'], multiline=False, custom_value_template=None), IOLine(key='release_year', type='int', rule=None, options=None, multiple_select=False, default=None, placeholder=['release_year'], multiline=False, custom_value_template=None)])

output_model = OutputModel(lines=[IOLine(key='genre', type='str', rule=None, options=['fiction', 'non-fiction', 'fantasy', 'sci-fi', 'romance', 'thriller', 'horror', 'history'], multiple_select=False, default=None, placeholder=['genre'], multiline=False, custom_value_template=None)])

In [12]:
output = generate_instruction_from_models(input_model, output_model)

for m, r in output:
    print(m)
    print(r)
    print("---")


Error while parsing output: mapping values are not allowed here
  in "<unicode string>", line 1, column 50:
     ... put model consists of three keys: 'book_title', 'book_author', a ... 
                                         ^
Failed Multiline parsing
Error while parsing output: mapping values are not allowed here
  in "<unicode string>", line 1, column 51:
     ... k takes an input with three keys: 'book_title', 'book_author', a ... 
                                         ^
Failed Multiline parsing
Based on the input/output model can you what the instruction of the task is?
Determine the genre of a book based on its title, author, and release year.
---
I have lost the instruction for that task. Can you help me work out what it was?
Extract the genre of the book based on the provided input.
---
Can you work out what the task should do based on the input/output model?
Determine the genre of the book based on the given 'book_title', 'book_author', and 'release_year' information.
---

In [3]:
output = generate_instruction(job.input_model, job.output_model, debug=True)

>>> Placeholder map:
 {'{input}': {'inp_model': "<'description': str>\n<'sources': list[str]>", 'out_model': "External types: <list[str], options=['numeric', 'textual', 'multimedia', 'code'], multiple_select=True, multiline=False>\nInternal types: <list[str], options=['numeric', 'textual', 'multimedia', 'code'], multiple_select=True, multiline=False>", 'input_object': None, 'output_object': None}}
>>> Formatted inputs:
 {'input': "Inp model: '<''description'': str>\n\n  <''sources'': list[str]>'\nOut model: 'External types: <list[str], options=[''numeric'', ''textual'', ''multimedia'',\n  ''code''], multiple_select=True, multiline=False>\n\n  Internal types: <list[str], options=[''numeric'', ''textual'', ''multimedia'', ''code''],\n  multiple_select=True, multiline=False>'\nInput object: null\nOutput object: null\n"}
>>> Input schema:
 <'input_model': any>
<'output_model': any>
<'example_input_object': any>
<'example_output_output': any>
>>> Input model:
 lines=[IOLine(key='input_model

In [5]:
from structgenie.engine import StructGenie
from structgenie.input_output import InputModel, OutputModel

def generate_instruction_from_models(
        input_model: InputModel,
        output_model: OutputModel,
        input_object: dict = None,
        output_object: dict = None,
        **kwargs):
    """Generate an instruction for a specific job."""
    template = """I have a task that takes an input and produces an output. Below you will find the input and output models for that task which define and restrict the possible input objects and the desired output object.
{meta_instruction}
Think step-by-step to reverse engineer the task instruction. What other inputs might look like with the mentioned keys? What is the relationship between them? What might be the right operation (label, extract, compress, generate, etc.) to get from any input object with that structure to the desired output object?


# Input
Input Model: {inp_model}
Output Model: {out_model}
---
Reasoning: <multiline>
Instruction: <multiline>
"""
    engine = StructGenie.from_template(template, prompt_kwargs={"set_format_tags": False}, **kwargs)
    
    meta_instructions = [
        "Based on the input/output model can you what the instruction of the task is?",
        "I have lost the instruction for that task. Can you help me work out what it was?",
        "Can you work out what the task should do based on the input/output model?",
        "When presented with this input/output model what would you expect the task to do?",
    ]
    
    results = []
    for meta in meta_instructions:
        result = engine.run(
            dict(
                meta_instruction=meta,
                inp_model=input_model.template_schema,
                out_model=output_model.template_schema,
                input_object=input_object,
                output_object=output_object,
            )
        )
        results.append((meta, result["instruction"]))
        

    return results

In [9]:
output = generate_instruction_from_models(input_model, output_model)

Error while parsing output: mapping values are not allowed here
  in "<unicode string>", line 1, column 48:
     ... input model consists of two keys: 'description' and 'sources'. ' ... 
                                         ^
Failed Multiline parsing
Error while parsing output: mapping values are not allowed here
  in "<unicode string>", line 1, column 48:
     ... input model consists of two keys: 'description' and 'sources'. ' ... 
                                         ^
Failed Multiline parsing
Error while parsing output: mapping values are not allowed here
  in "<unicode string>", line 1, column 48:
     ... input model consists of two keys: 'description' and 'sources'. ' ... 
                                         ^
Failed Multiline parsing
Error while parsing output: mapping values are not allowed here
  in "<unicode string>", line 1, column 48:
     ... input model consists of two keys: 'description' and 'sources'. ' ... 
                                         ^
Faile

"Create a task instruction that takes a description as input and a list of sources. The task instruction should have two types: external types and internal types. Both types should be a list of strings with the options of 'numeric', 'textual', 'multimedia', and 'code'. The external types should allow for multiple select and should not be multiline. The internal types should also allow for multiple select and should not be multiline."

In [10]:
for m, r in output:
    print(m)
    print(r)
    print("---")

Based on the input/output model can you what the instruction of the task is?
Transform the input object into a list of strings representing external and internal types.
---
I have lost the instruction for that task. Can you help me work out what it was?
Given a description and a list of sources as input, produce a list of external and internal types. The external and internal types can be selected from the options "numeric", "textual", "multimedia", and "code". The types can be selected multiple times but cannot be multiline.
---
Can you work out what the task should do based on the input/output model?
Given a description and a list of sources, generate a list of external and internal types.
---
When presented with this input/output model what would you expect the task to do?
Given a description and a list of sources, generate two lists: one for external types and one for internal types. Both lists should allow multiple selections from the options 'numeric', 'textual', 'multimedia', an

In [2]:
input_object = dict(
    book_title="The Lord of the Rings",
    book_author="J. R. R. Tolkien",
    release_year=1954,
)
output_object = dict(
    genre="fantasy",
)

instruction_from_working_out(input_object, output_object)

'Determine the genre of the book based on the given information.'

In [11]:
extend_instruction_by_model(instruction='Determine the genre of the book based on the given information.', input_model=input_model, output_model=output_model)

'Determine the genre of the book based on the given information. Consider the book title, author, and release year.'

In [3]:
input_object = dict(
    data_source=["internal"],
    product_description="A presentation for shareholders",
)
output_object = dict(
    internal_data_types=["financial", "textual"],
    external_data_types=[],
)

instruction_from_working_out(, output_object)

'Identify the internal data types for a presentation for shareholders.'

In [3]:
result = mutate_instruction('Extract the genre of the book.', num=2)
print(result)

['Retrieve the category of the novel.', 'Obtain the classification of the publication.', 'Extract the genre of the book.', 'Break down the book into its genre.', 'Describe the genre of the book, focusing on its themes, plot, and characters.', 'What is the genre of the book from the perspective of the author, critics, and readers?']


In [ ]:
template = """I gave a friend an instruction and some advice. They followed the instruction to get and advice to use the input and got the correct output.
Below are the correct examples of his workings out. Can you work out what the instruction was?
    
# Input
Working Out: <list, rule=(length=5)>
---
The Instruction was: <multiline>
"""

engine = StructGenie.from_template(template, debug=True)

work_outs = [{"input": input_object}]

In [15]:
template = """I have a task that takes an input and produces an output. Below you will find the input and output models for that task which define and restrict the possible input objects and the desired output object.
{meta_instruction}
Think step-by-step to reverse engineer the task instruction. What other inputs might look like with the mentioned keys? What is the relationship between them? What might be the right operation (label, extract, compress, generate, etc.) to get from any input object with that structure to the desired output object?


# Input
Input Model: {inp_model}
Output Model: {out_model}
---
Reasoning: <multiline>
Instruction: <multiline>
"""

engine = StructGenie.from_template(template, debug=True)
output_model = engine.output_model

In [16]:
output_model

OutputModel(lines=[IOLine(key='reasoning', type='str', rule=None, options=None, multiple_select=False, default=None, placeholder=['reasoning'], multiline=True, custom_value_template=None), IOLine(key='instruction', type='str', rule=None, options=None, multiple_select=False, default=None, placeholder=['instruction'], multiline=True, custom_value_template=None)])

# Multiline fix

In [18]:
from structgenie.parser.string import parse_multi_line_string

text = """
Reasoning: The task involves creating an instruction based on the input and output model provided. The input object consists of a description and a list of sources. The output object consists of two lists: external types and internal types. Both lists have the same options and properties.

Instruction: Create a task instruction that takes a description and a list of sources as input. The task should generate two lists as output: external types and internal types. Both lists should have the options 'numeric', 'textual', 'multimedia', and 'code'. The user should be able to select multiple options from both lists, and the input and output objects should not allow multiline input."""


if any(line.multiline for line in output_model.lines):
    idx_keys = [(i, line.key) for i, line in enumerate(output_model.lines) if line.multiline]
    key_pairs = []
    for idx, key in idx_keys:
        next_key = output_model.lines[idx + 1].key if idx + 1 < len(output_model.lines) else None
        key_pairs.append((key, next_key))
    try:
        print(key_pairs)
        result = parse_multi_line_string(text, key_pairs)
    except Exception as e:
        print("failed")

[('reasoning', 'instruction'), ('instruction', None)]
failed


In [24]:
from structgenie.parser.string import format_as_key
import re
def extract_multiline_text(text: str, key: str, next_key: str):
    if next_key:
        return re.search(f"\n{key}:(.*)\n{next_key}:", text, re.DOTALL).group(1).strip()
    return re.search(f"\n{key}:(.*)", text, re.DOTALL).group(1).strip()

def remove_multiline_section(text: str, key: str, next_key: str):
    if next_key:
        return re.sub(fr"{key}:(.|\n)*{next_key}:", f"{next_key}:", text, re.DOTALL)
    return re.sub(fr"{key}:(.|\n)*", "", text, re.DOTALL)


In [25]:
s = text
d = {}
for key, next_key in key_pairs:
    key_ = format_as_key(key)
    next_key_ = format_as_key(next_key) if next_key else None
    d[key] = extract_multiline_text(s, key_, next_key_)
    text = remove_multiline_section(text, key_, next_key_)

In [26]:
text

'\n'

In [28]:
if text.strip():
    print("failed")
    

In [1]:
job.instruction

NameError: name 'job' is not defined

In [3]:
job.instruction = "Transform the input object into a list of strings representing external and internal types."

In [4]:
from llmp.components.instruction.mutation import InstructionMutation

mutations = InstructionMutation(job).run()

Error while parsing output: mapping values are not allowed here
  in "<unicode string>", line 3, column 298:
     ... ld be a dictionary with two keys: "External types" and "Internal ... 
                                         ^
Failed Multiline parsing
Error while parsing output: while parsing a block mapping
  in "<unicode string>", line 1, column 1:
    Reasoning: "The task is to conve ... 
    ^
expected <block end>, but found '<scalar>'
  in "<unicode string>", line 3, column 174:
     ...  should be a dictionary with a "description" key and a "sources" ... 
                                         ^
Failed Multiline parsing
Error while parsing output: mapping values are not allowed here
  in "<unicode string>", line 1, column 78:
     ...  input object has two attributes: "description" (a string) and " ... 
                                         ^
Failed Multiline parsing


In [5]:
mutations

['Convert the given object into a collection of strings that represent both external and internal types. The input object should be a dictionary with a "description" key and a "sources" key, both of which should be strings. The output object should be a dictionary with two keys: "External types" and "Internal types". Both keys should have values that are lists of strings. The possible values for the lists are "numeric", "textual", "multimedia", and "code". The lists can have multiple selections and should not be multiline.',
 'Generate a list of strings that describe the external and internal types of the input object.',
 'Transform the input object into a list of strings representing external types. The input object should have a description string and a list of sources. The output object should be a list of strings representing external types, with options for numeric, textual, multimedia, and code types, and the ability to select multiple types.',
 'Transform the input object into a